(1) 데이터의 표현방식<br>
https://docs.google.com/document/d/1nro5Xpkex6PSg92uW7iruz4hi2wtrh6a6yISxvGCm24/edit?usp=sharing

(2) 데이터 읽어오기 (분할하여 불러오기)

(3) 4분 30초 : cost function에서 axis=1이 무슨 의미지? <br>

(4) Python 문법 중에 with ~ as 공부하기<br>

(5) cpoch(에포) / batch 이 용어들은 전체 데이터에서 x개씩 나눠서 학습시키는 개념에서 출발함.<br>
    Example : if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.<br>
        - batch size : 전체 데이터를 일정한 크기로 나눴을 때의 크기를 말함
        - epoch : 전체 데이터에서 n개로 나눠서 학습을 시킬 때 전체 training data가 1회 학습됐을 때 1 epoch라고 한다. 
        
(6) epoch가 늘어날수록 cost가 줄어드네.  >>> 음. 근데 데이터가 random으로 섞이지 않으면 epcoh가 늘어나더라도 의미없는거 아닌가? 같은 데이터로 학습시킨다고 뭐가 달라지나? 의견이 필요함

# 데이터 불러오기

In [69]:
import tensorflow as tf

In [70]:
from tensorflow.examples.tutorials.mnist import input_data

In [71]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# 경사하강을 위한 변수설정

In [72]:
nb_classes = 10

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, nb_classes])

X,Y의 데이터가 입력되는 행/열의 위치가 계속 헷갈려서 적는다.<br>
간단하다. 데이터 포인터(행)을 한 줄로 읽는다고 생각해보자. <br>
그때 입력해야할 공간을 종이에 그려넣어보자.<br>
위 코드에 따르면 훈련될 데이터(X)는 총 784개의 입력공간이 필요하다.<br>
정답 데이터(Y)는 원-핫 인코딩으로 표현됐으니까 총 10개의 입력공간이 필요하다!

In [73]:
W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

In [74]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

In [75]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

Corss entropy. 수식을 외우던지 적어두는 편이 낫겠다, 근데 axis = 1 무슨 의미?

In [76]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 점수측정을 위한 변수 생성

In [77]:
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))

Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead


is_correct : hypothes(예측값)와 Y(실제값)이 1로 서로 같은건 몇개니?

In [78]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

accuracy(정확도) : is_correct의 수를 n으로 나누면 몇이니? 

# 큰 용량의 데이터 효율적으로 불러오기& 점수측정

In [79]:
training_epochs = 15

epoch : 전체 데이터에서 n개로 나눠서 학습을 시킬 때 전체 training data가 1회 학습됐을 때 1 epoch라고 한다. <br>
즉, 전체 데이터를 15번 훈련시킨다는 의미이다.

In [80]:
batch_size = 100

batch size : 전체 데이터를 일정한 크기로 나눴을 때의 크기를 말함
 

In [81]:
with tf.Session() as sess:                  ## 파이썬 "with 문법"에 대해서 공부
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer],
                            feed_dict={X: batch_xs, Y:batch_ys})
            avg_cost += c / total_batch     ## 적절한 w가 갱신되면서 cost가 개선된다. >> cost가 줄면 갱신된 w를 채택, 아니면 가만히 두고..
            
        
        print('Epoch: ', '%04d' % (epoch +1),
                                 'cost =', '{:.9f}'.format(avg_cost))
    


Epoch:  0001 cost = 2.760421296
Epoch:  0002 cost = 1.116716174
Epoch:  0003 cost = 0.889598542
Epoch:  0004 cost = 0.777458741
Epoch:  0005 cost = 0.705288055
Epoch:  0006 cost = 0.654706435
Epoch:  0007 cost = 0.616288431
Epoch:  0008 cost = 0.585360908
Epoch:  0009 cost = 0.560097825
Epoch:  0010 cost = 0.538853061
Epoch:  0011 cost = 0.520745813
Epoch:  0012 cost = 0.505191663
Epoch:  0013 cost = 0.491388314
Epoch:  0014 cost = 0.479095412
Epoch:  0015 cost = 0.468179831


# 부록

with문 설명

순간순간 내용을 리셋하는 등의 작업할때 주로 사용되는 문법인거 같다.<br>
예를들어 한글문서를 작성중에 <새로 만들기> 같은 기능이랄까?<br>

In [84]:
with tf.Session() as sess:   
    sess.run(tf.global_variables_initializer())

tensorflow에서는 항상 Session을 초기화시켜서 사용한다고 했다.<br>
그래서 with를 사용해줬다.